In [1]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import praw

import pickle

from markdown import markdown


from bs4 import BeautifulSoup as bs
import os

In [2]:
import requests

In [3]:

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

In [4]:
def is_image_submission(s):
    url = s.url
    return 'imgur' in url or '.jpg' in url or '.png' in url

In [5]:
def ocr_submissions(s):
    url = s.url
    if 'imgur' in url and '.jpg' not in url:
        url = url + ".jpg"

    file_name = url.split("/")
    if len(file_name) == 0:
        file_name = re.findall("/(.*?)", url)
    file_name = os.path.join("img",file_name[-1])
    if "." not in file_name:
        file_name += ".jpg"
    r = requests.get(url)
    with open(file_name,"wb") as f:
        f.write(r.content)

    return pytesseract.image_to_string(Image.open(file_name))

In [6]:
pytesseract.pytesseract.tesseract_cmd = input()

D:\bin\tesseract\tesseract.exe


In [7]:
pytesseract.pytesseract.tesseract_cmd

'D:\\bin\\tesseract\\tesseract.exe'

In [8]:
reddit = praw.Reddit(client_id='vKDJ34lnNRAt5Q',
                     client_secret='ap0vMAKpJ7ukjRAEQLCGltyA-so',
                     password='thisisapassword',
                     user_agent='testscript by /u/throwawaytsds',
                     username='throwawaytsds')

In [9]:
print(reddit.user.me())


throwawaytsds


In [10]:
reddits = [
    "environment",
    "economics",
    "religion",
    "education",
    "immigration",
    "lgbt",
    "war",
    "feminism",
    "business",
    "firearms",
    "politics",
    "gay"
]

In [11]:
df = pd.DataFrame(columns=['text','category'])

In [12]:
n = 1000

In [13]:
os.mkdir("img")

In [ ]:
errs = []
for r in reddits:
    submisssions = reddit.subreddit(r).top('all',limit=n)
    for _ in tqdm(range(n)):
        try:
            s = submisssions.next()
            text = s.title
            try:
                if is_image_submission(s):
                    text += ocr_submissions(s)
            except Exception as e:
                errs.append(s)
            df.loc[len(df)] = [text, r]
        except StopIteration as e:
            pass
        except Exception as e:
            print(e)

In [40]:
print(len(errs))

153


In [41]:
for i, _ in df[df['category']=='gay'].iterrows():
    df.loc[i, 'category']='lgbt'

In [42]:
np.unique(df.category)

array(['business', 'economics', 'education', 'environment', 'feminism',
       'firearms', 'immigration', 'lgbt', 'politics', 'religion', 'war'],
      dtype=object)

In [43]:
for i, _ in df[df['category']=='business'].iterrows():
    df.loc[i, 'category']='economics'

In [44]:
np.unique(df.category)

array(['economics', 'education', 'environment', 'feminism', 'firearms',
       'immigration', 'lgbt', 'politics', 'religion', 'war'], dtype=object)

In [58]:
len(df)

11945

In [62]:
df.sample(10)

,text,category
17011,The real reason why homosexuality is considere...,lgbt
22684,Melania Trump arrives in Saudi Arabia without ...,politics
21217,Vice brings up a good point for once. Why is t...,firearms
19985,Instagram Will Now Tell You Who's Getting Paid...,economics
16990,LGBTQ for the winBryan Fischer @ :\n@BryanJFis...,lgbt
18495,The despondent battle for Hill 3234,war
17434,Shirtless kinda means a different thing now.. ...,lgbt
22470,Woman Dragged Out of West Virginia House Heari...,politics
14437,The head of Wisconsin's anti-gay Christian gro...,religion
20177,Free Trials Without The Hassle: Mastercard rul...,economics


In [64]:
c, v = np.unique(df.category, return_counts=True)
c, v

(array(['economics', 'education', 'environment', 'feminism', 'firearms',
        'immigration', 'lgbt', 'politics', 'religion', 'war'], dtype=object),
 array([1977, 1000,  995,  997,  991,  999, 1988,  999, 1000,  999],
       dtype=int64))

In [69]:
max_count = np.min(v)
max_count

991

In [75]:
new_df = pd.DataFrame(columns=['text','category'])

In [74]:
df = df.sample(frac=1).reset_index(drop=True)

In [77]:
for i, row in df.iterrows():
    cat = row.category
    if len(new_df[new_df.category==cat])<max_count:
        new_df.loc[len(new_df)] = [row.text, row.category]

In [78]:
c, v = np.unique(new_df.category, return_counts=True)
c, v

(array(['economics', 'education', 'environment', 'feminism', 'firearms',
        'immigration', 'lgbt', 'politics', 'religion', 'war'], dtype=object),
 array([991, 991, 991, 991, 991, 991, 991, 991, 991, 991], dtype=int64))

In [80]:
new_df.to_csv("08_new_df.pd")

In [63]:
df.to_csv("08_data.pd")

In [6]:
df = pd.read_csv("08_new_df.pd", index_col="Unnamed: 0")

In [7]:
df.sample(10)

,text,category
7276,Retired Miner With Black Lung Begs EPA To Save...,environment
7721,ICE Agents Arrested Over 200 People in Los Ang...,immigration
2521,Nearly Two-Thirds of Americans Can't Pass a Ba...,economics
1316,Tesla secures order from Wal-Mart for its new ...,economics
7564,A Democrat Ran on Climate Change in a Republic...,environment
3645,Help advice needed,immigration
8116,Elizabeth Warren Bill Would Mandate Companies ...,environment
9133,"Why poeple who take the Bible ""literally"" shou...",religion
8044,Netherlands says OK for biker gangs to fight I...,war
329,The student loan crisis is really an underempl...,education


In [8]:
len(df)

9910

# Preprocessing with spacy

In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [5]:
my_stop_words = [u'say', u'\'s', u'Mr', u'be', u'said', u'says', u'saying']
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

filtered = ['our', 'barbara','snowden','iowa','emanuel','goldman','bring','america','like','chrysler','jerry','voter','political','american','president','poll','independent','democratic','campaign','progressive','want','socialist','thing','think','vote','party', 'debate', 'primary', 'nomination', 'presidential', 'caucus', 'history', 'candidate','bernie','sanders','hillary','clinton','barack','obama','donald','trump','republican','democrat','republicans','democrats','romney']
filtered = [nlp(w)[0].lemma_ for w in filtered]
print(filtered)
def is_filtered(w):
    return str(w) in filtered

['-PRON-', 'barbara', 'snowden', 'iowa', 'emanuel', 'goldman', 'bring', 'america', 'like', 'chrysler', 'jerry', 'voter', 'political', 'american', 'president', 'poll', 'independent', 'democratic', 'campaign', 'progressive', 'want', 'socialist', 'thing', 'think', 'vote', 'party', 'debate', 'primary', 'nomination', 'presidential', 'caucus', 'history', 'candidate', 'bernie', 'sander', 'hillary', 'clinton', 'barack', 'obama', 'donald', 'trump', 'republican', 'democrat', 'republican', 'democrat', 'romney']


In [6]:
df_processed = pd.DataFrame(columns=['text','category'])

In [10]:
for i, row in tqdm(df.iterrows(),total=len(df)):
    text = row['text']

    texts_nlp, article_nlp, skl_texts = [], [], []

    phrase_nlp = nlp(text)
    for w in phrase_nlp:
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and w.is_alpha:
            if not is_filtered(w.lemma_) and len(w.lemma_) > 2:
                article_nlp.append(w.lemma_)

    df_processed.loc[len(df_processed)] = [' '.join(article_nlp), row['category']]

In [11]:
df_processed.sample(10)

,text,category
10280,bill gates and other billionaires backing nucl...,business
9882,fox employees walk eggshells heavy layoffs con...,business
6623,high school sweetheart celebrate year cartagen...,lgbt
7963,israel launch major strike target syria follow...,war
1967,judge exxon pay violate clean air act time,environment
10143,how got hepatitis drug for buy from india,business
3306,the rich world need high real wage growth,economics
901,science mantra administration reveal lobbyist ...,environment
3360,chinese pour real estate study investment set ...,economics
9056,female overwatch player harass minute upload y...,feminism


In [12]:
df_processed.to_csv("08_data_processed.pd")